In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.impute import KNNImputer
#from missforest import MissForest
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap
from sklearn.manifold import trustworthiness
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.mixture import GaussianMixture


In [39]:
path = "/scratch/haniel.botelho/physionet.org/files/eicu-crd/2.0/"


df_patient = pd.read_csv(path + 'patient.csv')

In [40]:
df_patient.columns

Index(['patientunitstayid', 'patienthealthsystemstayid', 'gender', 'age',
       'ethnicity', 'hospitalid', 'wardid', 'apacheadmissiondx',
       'admissionheight', 'hospitaladmittime24', 'hospitaladmitoffset',
       'hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 'hospitaldischargeoffset',
       'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
       'admissionweight', 'dischargeweight', 'unitdischargetime24',
       'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',
       'uniquepid'],
      dtype='object')

In [41]:
df_patient = df_patient[['patientunitstayid','unitdischargeoffset','unitdischargestatus']]

In [42]:
df_gmm_intersecao = pd.read_csv('final_clusters_gmm_intersecao.csv')
df_gmm = pd.read_csv('final_clusters_gmm.csv')

In [43]:
df_gmm_intersecao = pd.merge(df_patient, df_gmm_intersecao, on='patientunitstayid')
df_gmm = pd.merge(df_patient, df_gmm, on='patientunitstayid')

In [44]:
df_gmm = df_gmm[['patientunitstayid','unitdischargeoffset','unitdischargestatus','Cluster']]
df_gmm_intersecao = df_gmm_intersecao[['patientunitstayid','unitdischargeoffset','unitdischargestatus','Cluster']]

In [45]:
df = pd.read_csv('./eicu/data_imputed.csv')

In [46]:
df['Gender'] = df[['gender_Female', 'gender_Male', 'gender_Unknown']].idxmax(axis=1)
df['Gender'] = df['Gender'].str.replace('gender_', '')

# Combina as colunas de etnia em uma única coluna
df['Ethnicity'] = df[['ethnicity_African American','ethnicity_Asian','ethnicity_Caucasian','ethnicity_Hispanic','ethnicity_Native American','ethnicity_Other/Unknown']].idxmax(axis=1)
df['Ethnicity'] = df['Ethnicity'].str.replace('ethnicity_', '')

In [47]:

df_gmm = pd.merge(df, df_gmm, on='patientunitstayid')
df_gmm_intersecao = pd.merge(df, df_gmm_intersecao, on='patientunitstayid')

In [48]:
df_gmm.drop(columns=['ethnicity_African American','ethnicity_Asian','ethnicity_Caucasian','ethnicity_Hispanic','ethnicity_Native American','ethnicity_Other/Unknown'],inplace=True)
df_gmm_intersecao.drop(columns=['ethnicity_African American','ethnicity_Asian','ethnicity_Caucasian','ethnicity_Hispanic','ethnicity_Native American','ethnicity_Other/Unknown'],inplace=True)

In [49]:
df_gmm.drop(columns=['gender_Female','gender_Male'],inplace=True)
df_gmm_intersecao.drop(columns=['gender_Female','gender_Male'],inplace=True)

In [50]:
df_gmm

,patientunitstayid,age,bands,CRP,ESR,temp,lactate,WBC,lymphs,resp,...,calcium,phosphate,magnesium,sofascore,gender_Unknown,Gender,Ethnicity,unitdischargeoffset,unitdischargestatus,Cluster
0,141194,68.0,18.158455,11.812137,63.724572,37.114019,1.900000,4.3,1.000000,25.000000,...,6.9,3.135049,1.200000,4.794150,False,Male,Caucasian,4813,Alive,0
1,141197,71.0,8.415707,18.270326,56.769065,36.989138,1.714293,29.5,3.000000,24.000000,...,9.0,2.899982,1.884892,9.000000,False,Male,Caucasian,74,Alive,0
2,141227,82.0,10.000000,36.656951,63.106964,37.017340,4.300000,42.7,7.000000,25.470210,...,8.3,3.582903,1.900000,4.000000,False,Male,Caucasian,1652,Alive,0
3,141266,73.0,9.445528,106.207760,50.837647,36.736394,4.500000,5.7,8.000000,19.000000,...,8.5,2.200000,1.900000,3.000000,False,Male,Caucasian,1501,Alive,0
4,141288,61.0,26.000000,103.569541,49.337575,37.216494,3.200000,6.9,2.000000,25.998868,...,8.7,2.600000,2.000000,4.000000,False,Female,Caucasian,1631,Alive,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30003,3352922,77.0,16.858081,-2.322198,49.345955,36.697581,7.000000,3.8,1.000000,19.000000,...,4.6,2.200000,1.700000,4.683544,False,Male,Caucasian,5096,Alive,1
30004,3352940,23.0,16.164344,55.052404,54.001731,37.101823,4.000000,5.7,7.000000,55.000000,...,7.5,5.100000,1.500000,5.910140,False,Female,Caucasian,16894,Alive,1
30005,3352961,70.0,10.765257,7.812510,56.788690,37.067962,1.300000,9.4,10.000000,29.000000,...,7.8,2.700000,1.500000,5.028040,False,Male,Caucasian,1473,Alive,0
30006,3353145,51.0,11.690899,32.454674,62.238027,37.098182,2.624407,28.4,4.676758,27.000000,...,8.8,2.500000,1.500000,5.123276,False,Male,African American,14565,Alive,0


In [51]:
df_gmm.to_csv('analiseGMM.csv',index=False)
df_gmm_intersecao.to_csv('analiseGMM_intersecao.csv',index=False)